In [1]:
import os
import json
import requests
import re
import numpy as np
import faiss
import pickle
import gzip
from bs4 import BeautifulSoup
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai
from openai import OpenAI # API v2.0
#from openai.embeddings_utils import get_embedding, get_embeddings # before v1.2

with open("openai_key.txt", "r") as f:
    openai_key = f.read().strip()
    openai_client = OpenAI(api_key=openai_key)

In [4]:
# 定义函数

def crawler_page_urls():
    """# 爬取所有2866个连接 """
    urls0 = [f"https://gwins.org/cn/milesguo/list_2_{i}.html" for i in range(1,73)]
    urls1 = []
    for url in tqdm(urls0):
        response = requests.get(url)
        if response.status_code == 200:    
            soup = BeautifulSoup(response.content, 'html.parser')
            html_doc = soup.get_text() 
            link_string = '\n'.join([str(link) for link in soup.find_all('a')])
            pattern = r"/cn/milesguo/[\w/]+\.html"
            matches = re.findall(pattern, link_string)
            urls2 = [f"https://gwins.org{x}" for x in matches]
            urls1 += urls2
            print(len(urls1))
        else:
            print(f"无法获取页面{url}，HTTP状态码：{response.status_code}")
    return urls1

def download_documents():
    """# 爬取所有2866个文章，并保存为文档""" 
    urls1 = crawler_page_urls() # 爬取所有2866个连接
    out_folder = "./txts"
    if not os.path.isdir(out_folder): 
        os.mkdir("./txts")
    for url in tqdm(urls1):
        pattern = r'\d+'
        id = re.search(pattern, url).group() # 获取网页编号
        response = requests.get(url)
        if response.status_code == 200:    
            soup = BeautifulSoup(response.content, 'html.parser')
        else:
            print(f"无法获取页面{url}，HTTP状态码：{response.status_code}")
            continue
        html_doc = soup.get_text()  # 获取网页中的纯文本内容
        html_doc = re.sub(r'\s+', ' ', html_doc) # 去掉多余空格
        
        file_path = os.path.join(out_folder, f"{id}.txt")
        with open(file_path, "w") as f:
            f.write(html_doc) # 保存

In [86]:
def extract_titles(input_dir="./txts/", out_file="titles.json"):
    """提取每个文档的标题"""
    files = [os.path.join(input_dir, x) for x in os.listdir(input_dir)]
    dict1 = dict()
    for in_file in tqdm(files):
        id = os.path.basename(in_file).split(".")[0]
        with open(in_file, "r") as f:
            txt = f.read()
        pattern1 = r'^(.*?)首页'
        match = re.match(pattern1, txt)
        if match: 
            title = match.group(1)
            title = title.replace("\n", " ")
        else:
            title = "unknown title"
        dict1[id] = title
        #print(title)
    with open(out_file, "w") as f:
        json.dump(dict1, f)

def load_titles(file="title.json"):
    with open(file, "r") as f:
        dict1 = json.load(f)
    return dict1
    
def load_data_to_paragraphs(file):
    """ 将长文档分解成1000字以内短文档. 
    因为openai sentence embedding ada 002 8000 input token, 最大2000汉字
    但是逼近2000后语义编码效果会下降
    """
    with open(file, "r") as f:
        data = f.read()
    pattern1 = r'^.*?内容梗概: '
    pattern2 = r' 友情链接：Gnews \| Gclubs \| Gfashion \| himalaya exchange \| gettr \| 法治基金 \| 新中国联邦辞典 \| $'
    data = re.sub(pattern1, "", data)
    data = re.sub(pattern2, "", data)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    txts = text_splitter.split_text(data)
    return txts

def sentence_embedding_batch(txts, id):
    """将list of text编码为sentence embedding 1538维"""
    l1 = []
    #embs = get_embeddings(txts, engine="text-embedding-ada-002")
    response = openai_client.embeddings.create(input = txts, model="text-embedding-ada-002")
    response = json.loads(response.json())["data"]
    embs = [x["embedding"] for x in response]
    for i, txt in enumerate(txts):
        label = f"{id}-{i}"
        emb = embs[i]
        l1.append((label, txt, emb))
    return l1

def encoding_file(in_file, output_dir):
    """将文档.txt文件编码为同名 embedding文件"""
    id = os.path.basename(in_file).split(".")[0]
    out_file = os.path.join(output_dir, id+".npz")
    txts = load_data_to_paragraphs(in_file)
    packs = sentence_embedding_batch(txts, id)
    serialized_data = pickle.dumps(packs)
    compressed_data = gzip.compress(serialized_data)
    with open(out_file, "wb") as file:
        file.write(compressed_data)

def encoding_files(input_dir = "./txts/", output_dir = "./emb"):
    """批量将文件夹下txt文件编码为同名 embedding文件，openai 3美元"""
    files = [os.path.join(input_dir, x) for x in os.listdir(input_dir)]
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    for i, in_file in enumerate(tqdm(files)):
        encoding_file(in_file, output_dir)

def decoding_file(file):
    with open(file, "rb") as f:
        compressed_data = f.read()
    decompressed_data = gzip.decompress(compressed_data)
    l1 = pickle.loads(decompressed_data)
    return l1

def build_faiss_index(embs):
    import faiss
    d = 1536
    nlist = 100
    index = faiss.IndexFlatIP(d)
    #index = faiss.IndexIVFFlat(index, d, nlist)
    index.train(embs)
    index.add(embs)
    return index
    

def build_veactor_search_index(folder="./emb"):
    files = [os.path.join(folder, x) for x in os.listdir(folder)]
    dict1 = dict()
    i = 0
    embs = []
    for file in tqdm(files):
        l1 = decoding_file(file)
        for idx, txt, emb in l1:
            dict1[i] = {"idx": idx, "txt": txt, "emb": emb}
            embs.append(emb)
            i+=1
    embs = np.vstack(embs)
    embs /= np.linalg.norm(embs, ord=2, axis=-1, keepdims=True) + 1e-8
    faiss_index = build_faiss_index(embs)
    return embs, dict1, faiss_index

def text_search(query, faiss_index, dict_emb, dict_title, k=3):
    """ 每1000 token 250汉字 0.0001美元"""
    if len(query)<10:
        query = f"这是一个关于{query}的句子"
    #emb_query = get_embedding(query, engine="text-embedding-ada-002")
    response = openai_client.embeddings.create(input=[query], model="text-embedding-ada-002")
    emb_query = json.loads(response.json())["data"][0]["embedding"]
    emb_query = np.array(emb_query).reshape((1, -1))
    D, I = faiss_index.search(emb_query, k)
    txts = []
    for i in I[0]:
        idx = dict_emb[i]["idx"]
        idx0 = idx.split("-")[0]
        txt = dict_emb[i]["txt"]
        title = dict_title[idx0]
        txts += [(title, txt)]
    return txts

In [60]:
#download_documents()
#encoding_files(input_dir = "./txts/", output_dir = "./emb")
#extract_titles(input_dir="./txts/", out_file="titles.json")
embs, dict_emb, faiss_index = build_veactor_search_index(folder="./emb")
dict_title = load_titles(file="titles.json")

100%|██████████████████████████████████████████████████████████████████████████████| 2866/2866 [00:05<00:00, 559.84it/s]


In [91]:
txt_query = "接种辉瑞疫苗有哪些后果？"
txts_retrival = text_search(txt_query, faiss_index, dict_emb, dict_title, k=3) #请求10000次API成本1美金
txts_retrival

[(' 郭文贵2021年10月29日直播 20211029_1格芯在美国上市的重大意义和其未来价值，格芯与数字货币紧密相连；喜币周一上市会带来什么样的改变；房地产会在明年5-7月爆雷，是中共制造的最大的噩梦 ',
  '（新冠疫苗）有许多不良反应，很多副作用包括肌肉疼痛、胸痛、呼吸困难，我们看到肺部症状甚至肺炎。 主持人：心脏炎症？ 嘉宾：心脏炎症、肺部炎症。 主持人：有时候会很致命。 嘉宾：会的。 嘉宾：有趣的是我们有4个孩子因为心肌炎来到诊所。 主持人：他们打疫苗了吗？ 嘉宾：都打了疫苗，12-16岁已经批准要求接种，我们那里有很多孩子为了回到运动场，所以他们开始接种疫苗。我们有4例心肌炎。 主持人：好， 那现在的问题是这个年龄段通常会发生心肌炎吗？你是每年都看到几例呢？还是你将其归咎于疫苗？ 嘉宾：在我将近20年的职业生涯中直到今年我才在青少年或儿童中看到心肌炎的病例。 主持人：你现在看到4例。 嘉宾：是的今年看到4例。 ● 10:43视频六：疫苗接种者讲述自己接种后各种副作用情况 我叫Kristy Dobbs 我是一名洗牙师，妻子和四个孩子的母亲，我支持科学我信任好的医疗。我在2021年1月18日 接受了第一剂也是唯一一剂 辉瑞公司的新冠疫苗，当时我在指定接种疫苗的诊所就立即出现了反应，刚接种后我的最初反应是在左臂上有一种刺痛的感觉，感觉像有水在我的手臂内滴落，我还没来得及坐下休息，一些反应就突然同时出现了，我无法呼吸 感觉很热，脉搏呼吸和心率都增加了，血压读数也高到了中风的程度，在注射辉瑞疫苗后的两天我的症状包括手臂酸痛、疲劳、淋巴结肿大和头痛，这些都是我预料到的，也是知情同意中有提到的正常副作用，然而在接种后的第三天疫苗的药效开始蹂躏我的身体，我的左肩胛区 出现了尖锐的刺痛以及我的左臂和左手出现了麻痹感和颤抖，到了第四天 我出现了全身震颤和麻痹。 以及体内像过电般的抽搐，感觉到极度疲劳 脑雾 肌肉疼痛和虚弱，无法入睡，出现多种自主神经功能紊乱，我有超过22种不同的症状，已经困扰我九个多月，至今我已经看了超过15个不同的医疗机构和专家早在3月份 我甚至与Safavi医生 （就职于于美国国立卫生研究院）进行了一次远程医疗访问，我被明确告知不要给我的孩子接种疫苗，我已经把我的血样送到了美国国立卫生研究院以及多个著名的大学和私人研究人员那里研究。 我的疫苗伤害已

In [89]:
def RAG_chatbot(txt_query, txts_retrival):
    """
    遇到与chatgpt观念不符合的文本时效果仍然不好，甚至会被掐断。
    需要试验其他本地化开源LLM
    
    """
    prompt = "\n\n".join([f"标题：{title}\n 正文：{txt}" for title, txt in txts_retrival])
    prompt = f"先摘要并总结参考文本，再解答这个问题{txt_query} 以下是参考文本\n\n{txts_retrival}"
    
    response = openai_client.chat.completions.create(model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": prompt}])
    txt_response = json.loads(response.json())["choices"][0]["message"]["content"]
    return txt_response

txt_query = "接种辉瑞疫苗有哪些后果？"
txts_retrival = text_search(txt_query, faiss_index, dict_emb, dict_title, k=3)
txt_response = RAG_chatbot(txt_query, txts_retrival)
print(txt_response)
print(txts_retrival)

根据参考文本中的内容，摘要如下：

1. 文本中提到了关于辉瑞疫苗的不良反应和副作用，包
[(' 郭文贵2021年10月29日直播 20211029_1格芯在美国上市的重大意义和其未来价值，格芯与数字货币紧密相连；喜币周一上市会带来什么样的改变；房地产会在明年5-7月爆雷，是中共制造的最大的噩梦 ', '（新冠疫苗）有许多不良反应，很多副作用包括肌肉疼痛、胸痛、呼吸困难，我们看到肺部症状甚至肺炎。 主持人：心脏炎症？ 嘉宾：心脏炎症、肺部炎症。 主持人：有时候会很致命。 嘉宾：会的。 嘉宾：有趣的是我们有4个孩子因为心肌炎来到诊所。 主持人：他们打疫苗了吗？ 嘉宾：都打了疫苗，12-16岁已经批准要求接种，我们那里有很多孩子为了回到运动场，所以他们开始接种疫苗。我们有4例心肌炎。 主持人：好， 那现在的问题是这个年龄段通常会发生心肌炎吗？你是每年都看到几例呢？还是你将其归咎于疫苗？ 嘉宾：在我将近20年的职业生涯中直到今年我才在青少年或儿童中看到心肌炎的病例。 主持人：你现在看到4例。 嘉宾：是的今年看到4例。 ● 10:43视频六：疫苗接种者讲述自己接种后各种副作用情况 我叫Kristy Dobbs 我是一名洗牙师，妻子和四个孩子的母亲，我支持科学我信任好的医疗。我在2021年1月18日 接受了第一剂也是唯一一剂 辉瑞公司的新冠疫苗，当时我在指定接种疫苗的诊所就立即出现了反应，刚接种后我的最初反应是在左臂上有一种刺痛的感觉，感觉像有水在我的手臂内滴落，我还没来得及坐下休息，一些反应就突然同时出现了，我无法呼吸 感觉很热，脉搏呼吸和心率都增加了，血压读数也高到了中风的程度，在注射辉瑞疫苗后的两天我的症状包括手臂酸痛、疲劳、淋巴结肿大和头痛，这些都是我预料到的，也是知情同意中有提到的正常副作用，然而在接种后的第三天疫苗的药效开始蹂躏我的身体，我的左肩胛区 出现了尖锐的刺痛以及我的左臂和左手出现了麻痹感和颤抖，到了第四天 我出现了全身震颤和麻痹。 以及体内像过电般的抽搐，感觉到极度疲劳 脑雾 肌肉疼痛和虚弱，无法入睡，出现多种自主神经功能紊乱，我有超过22种不同的症状，已经困扰我九个多月，至今我已经看了超过15个不同的医疗机构和专家早在3月份 我甚至与Safavi医生 （就职于于美国国立卫生研究院）进行了一次远程医疗访问，我被明确告知不要给我的孩子接种疫苗，我已经把我的

Distances: [[0.99999994 0.9173957  0.90672827 0.9045879  0.9038532  0.897203
  0.89675575 0.8966185  0.89628506 0.89613855]]
Indices: [[    0     1  5983 21589 24700 29149 19581 18945 14504 27710]]


In [94]:
import langchain
langchain.__version__

'0.0.334'